In [1]:
import pandas as pd
from Acesso import Login
from Query import Query
from glob import glob

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

In [2]:
arquivos=glob('*.xlsx')

df=pd.read_excel(arquivos[-1])

In [3]:
df

,SKU,Produto,Qtde Caixa,B,C,D,E,F,FUNC,B2C,...,DISTRIBUIDORES,Dif.1,TABELA FUNCIONÁRIO,Dif.2,TABELA CONSUMIDOR FINAL,Dif.3,Desc C,Desc D,Desc E,Desc F
0,1000,"CONGELADO ANEL PRE FORMADO CEBOLA MCCAIN 1,05KG",10,34.22,32.76,32.30,31.85,31.41,25.20,40.23,...,31.32,-0.98,24.44,-0.76,39.02,-1.21,0.0427,0.0561,0.0693,0.0821
1,1058,"CONG. BATATA MCCAIN SMILES NAC (1,5KG)",6,38.54,36.43,35.94,35.46,34.99,29.23,45.08,...,34.86,-1.08,28.35,-0.88,43.73,-1.35,0.0547,0.0675,0.0799,0.0921
2,1095,CONGELADO BATATA MCCAIN SURECRISP 9MM NAC,6,54.78,52.50,51.78,51.08,50.40,41.54,64.07,...,47.81,-3.97,38.35,-3.19,59.15,-4.92,0.0416,0.0548,0.0675,0.0800
3,9574,CONGELADO PICKERS ANEL DE CEBOLA INTEGRO MCCAI...,6,69.42,66.44,65.51,64.60,63.71,51.11,81.60,...,60.94,-4.57,47.55,-3.56,75.91,-5.69,0.0429,0.0563,0.0694,0.0823
4,9688,QUEIJO AMERICAN CHEESE PICANTE PROCESSADO FATI...,24,25.87,23.80,23.49,23.18,22.88,19.62,30.25,...,23.49,0.00,19.62,0.00,30.25,0.00,0.0800,0.0920,0.1040,0.1156
5,9697,CONGELADO MANDIOCA TOLETE DM 1.020KG,10,18.52,16.87,16.14,15.69,15.48,12.42,22.16,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0891,0.1285,0.1528,0.1641


In [9]:
#00000029,00000045,00000004,00000050

#alterar desconto

codigos=df['SKU'].unique().tolist()

conecta=sql.ConexaoSQL()

for c in codigos:
    
    try:
    
        desc=df.loc[df['SKU']==c,'Desc F'].max()
        
        querys=f"""

        UPDATE preco
        SET desc_prom={float(desc)},
        desc_max_prd={float(desc)}
        WHERE cd_prod={c} AND LTRIM(RTRIM(cd_tabela))='00000029'

        """

        sql.Salvar(querys,conecta)
        
        pass
    
    except Exception as erro:
        
        print(erro)
        
        pass
    
    #break
    
    pass


In [4]:
#alteração de preço massivo
tabelas_dict={'00000029':'B','00000045':'D','00000004':'FUNC','00000050':'B2C','TABA':'A'}

codigos=df['SKU'].unique().tolist()

conecta=sql.ConexaoSQL()

for cod,tab in tabelas_dict.items():
    
    for c in codigos:
        
        preco=df.loc[df['SKU']==c,tab].max()
        
        #Unid. VDA
        unid=df.loc[df['SKU']==c,'Unid. VDA'].max()
        
        #caixa Qtde Caixa
        caixa=df.loc[df['SKU']==c,'Qtde Caixa'].max()
        
        preco=preco if unid!='CX' else round(preco/caixa,2)

        querys=f"""

        UPDATE preco
        SET vl_preco={float(preco)}
        WHERE cd_prod={c} AND LTRIM(RTRIM(cd_tabela))='{cod}'

        """

        sql.Salvar(querys,conecta)

        #break

        pass
    
    print(f'Tabela {tab} alterada com sucesso!')
    
    pass


Tabela B alterada com sucesso!
Tabela D alterada com sucesso!
Tabela FUNC alterada com sucesso!
Tabela B2C alterada com sucesso!
Tabela A alterada com sucesso!
